# **MEF Metric Calculator - Looped**
## **High Resolution Sea Ice Diagsnostics Toolset**
 Cameron Carver - University of Cape Town - Jan 2025

This script produces a single netcdf file that includes the MEF metric data for all defined models listed.<br>
Model data is accessed through Pangeo then preprocesssed through xmip and reprojected using xesmf.<br>
The MEF metric is then calculated accordingly and saved to a single netcdf file.

### 1. Import all relevant packages

In [1]:
import os  
# os.chdir('') # User can provide root directory information here to simplify directory path definitions elsewhere 
import warnings
import copy
import time
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
from clisops.core.subset import subset_bbox  # For subsetting
import numpy as np
import matplotlib.colors as mcolors
from mpl_toolkits.basemap import Basemap
import intake
from xmip.preprocessing import combined_preprocessing
from xmip.utils import google_cmip_col
from xmip.postprocessing import match_metrics

### 2. Import Reference Dataset
Import the ancillary file of the observational data. <br>
Data Access https://nsidc.org/data/g02202/versions/4 <br>
Ancillary File: https://noaadata.apps.nsidc.org/NOAA/G02202_V4/ancillary/<br>
Provides the reference grid of which model data will be reprojected onto.<br>

Define landmask from ancillary file

In [2]:
access_ref_data = 'OBSERVATIONS/SICONC/G02202-cdr-ancillary-sh.nc' # Define location of locally stored grid reference file
ref_ds = xr.open_dataset(access_ref_data)

mask = xr.where(ref_ds.landmask > 0, float('nan'), 1)

### 3. Import Observational Data
Import the monthly observational data. <br>
Data Access https://nsidc.org/data/g02202/versions/4 <br>
Monthly Data https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/monthly/ <br>
Monthyl data was combined into a single NetCDF file along the time dimension.
#### Comparing model data to this data will be used to generate the MEF metric.
The data is sliced to a 36 year period of Dec 1978 - Nov 2014. <br>
All flagged values are masked out.<br>
Observational values of less than 15 are removed.<br>
Any instances where ice is observed for <= 20 occurances throughout the 432 months are masked out. <br>

The temporal mean is defined for all points on the grid.

In [3]:
access_obs_data='OBSERVATIONS/SICONC/seaice_conc_monthly_sh_n07_v04r00_all.nc' # Define location of locally stored observational reference dataset
obs_ds = xr.open_dataset(access_obs_data)                   # Open dataset
obs_ds = obs_ds.sel(tdim=slice('1978-12','2014-11'))  # Time slice of data
obs_da = obs_ds.cdr_seaice_conc_monthly                     # Select desired variable

obs_da = obs_da.where(obs_da <= 2.51, other=float('nan'))   # Set all flagged values to nan
count = obs_da.where(obs_da>0).count('tdim') # Count number of measurements present at each given point over entire time series
mincount = xr.where(count <= 20, float('nan'), 1) #Identify at which points less than 20 observational datapoints exist
obs_da = obs_da * mincount # If less than 20 obs then mask out
   
o_mean = obs_da.mean(dim='tdim')       # Temporal mean of observational product

### 3b. Varations
Uncomment the line of the respective variation mask of interest. <br>This isolates the observational datasets to produce MEF values for the gridpoints with the specified range of observed values.<br> 
**L15**:  Exclude [0,15] <br>
**ZERO**: Isolate 0 <br>
**I015**: Isolate (0,15]<br>
**I1580**: Isolate (15,80]<br>
**I80100**: Isolate (80,100]<br>

In [4]:
## When excluding values, mean must be taken afterwards
# obs_da = obs_da.where(obs_da > 0.15, other=float('nan'))                    #Exclude (-inf,15]           L15
# obs_da = obs_da.where(obs_da == 0, other=float('nan'))                      # Only values = 0%           ZERO
# obs_da = obs_da.where((obs_da > 0) & (obs_da <= 0.15), other=float('nan'))  # Only values = 0-15%        I015
# obs_da = obs_da.where((obs_da > 0.15) & (obs_da <= 0.8), other=float('nan'))# Only values = 15-80%       I1580
# obs_da = obs_da.where((obs_da > 0.80) & (obs_da <= 1), other=float('nan'))  # only values = 80-100%      I80100

### 4a. Define all CMIP models of interest
Comment/uncomment desired list of models to call

In [5]:
# # Historical CMIP Models
# mods = [
# 'GFDL-CM4',        #GOOD
#  # 'IPSL-CM6A-LR',      #CRASH X
#  'MIROC6',         #GOOD
#  'SAM0-UNICON',    #GOOD
#  'CanESM5',        #GOOD
#  'MPI-ESM-1-2-HAM',#GOOD
#  'NESM3',          #GOOD
#  'CAMS-CSM1-0',    #GOOD
#  # 'MPI-ESM1-2-HR',     #CRASH 
#  'MPI-ESM1-2-LR',  #GOOD
#  'GFDL-ESM4',      #GOOD
#  'NorESM2-LM',     #GOOD
#  'MRI-ESM2-0',     #GOOD* double lat/lon warning
#  'FGOALS-f3-L',    #GOOD* lat/lon warning w/xmip
#  'NorESM2-MM',     #GOOD
#  'FIO-ESM-2-0',    #GOOD
#  'BCC-CSM2-MR',    #GOOD
#  'BCC-ESM1',       #GOOD
#  'CMCC-CM2-SR5',   #GOOD
#  # 'EC-Earth3-AerChem',  #CRASH
#  'TaiESM1',        #GOOD
#  'NorCPM1',        #GOOD
#  # 'IPSL-CM5A2-INCA',   #CRASH X
#  'ACCESS-ESM1-5',  #GOOD
#  'ACCESS-CM2',     #GOOD
#  # 'CMCC-CM2-HR4',      #CRASH X
#  # 'EC-Earth3',         #CRASH X
#  # 'EC-Earth3-Veg-LR',  #CRASH
#  # 'EC-Earth3-Veg',     #NOT IN CAT1
#  'CAS-ESM2-0',      #GOOD
#  'FGOALS-g3',       #GOOD
#  # 'EC-Earth3-CC',      #CRASH
#  'CMCC-ESM2',       #GOOD
#  # 'IPSL-CM6A-LR-INCA'  #CRASH
# ]

### 4b. Define all HighResMIP models of interest

In [6]:
# HighResMIP Model List CAT1
mods = [
    'CMCC-CM2-HR4',    #GOOD
    'CMCC-CM2-VHR4',   #GOOD
    'HadGEM3-GC31-HM', #GOOD
    'HadGEM3-GC31-LL', #GOOD
    'HadGEM3-GC31-MM', #GOOD
    # 'CESM1-CAM5-SE-HR', #failed to load
    # 'CESM1-CAM5-SE-LR', #failed to load
    # 'HadGEM3-GC31-HH', #CRASH
    'GFDL-CM4C192', ]  #GOOD

### 5. Define Pangeo Catalog to call
Comment/uncomment lines to swap catalogs

In [7]:
url = "https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog.json" # Only stores that pass current tests
# url = "https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog_noqc.json" # Only stores that fail current tests
# url = "https://storage.googleapis.com/cmip6/cmip6-pgf-ingestion-test/catalog/catalog_retracted.json" # Only stores that have been retracted by ESGF
cmip6 = intake.open_esm_datastore(url)    ## HighResMIP catalogs, must uncomment one of above

# cmip6 = google_cmip_col()

### 6. Call list of models from catalog

In [8]:
# get the list of CMIP6 historical simulations with variable siconc
cat = cmip6.search(
    source_id=mods[:],
    variable_id='siconc', 
    table_id='SImon',
    # experiment_id='historical', # Uncomment for historical CMIP Models
    activity_id='HighResMIP',  # Uncomment for HighResMIP Models
    member_id='r1i1p1f1',
    grid_label='gn'
)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,sub_experiment_id,variant_label,version,zstore
0,HighResMIP,CMCC,CMCC-CM2-VHR4,hist-1950,r1i1p1f1,SImon,siconc,gn,none,r1i1p1f1,v20200917,gs://cmip6/cmip6-pgf-ingestion-test/zarr_store...
1,HighResMIP,CMCC,CMCC-CM2-HR4,hist-1950,r1i1p1f1,SImon,siconc,gn,none,r1i1p1f1,v20200917,gs://cmip6/cmip6-pgf-ingestion-test/zarr_store...
2,HighResMIP,MOHC,HadGEM3-GC31-HM,hist-1950,r1i1p1f1,SImon,siconc,gn,none,r1i1p1f1,v20180730,gs://cmip6/cmip6-pgf-ingestion-test/zarr_store...
3,HighResMIP,MOHC,HadGEM3-GC31-LL,hist-1950,r1i1p1f1,SImon,siconc,gn,none,r1i1p1f1,v20170921,gs://cmip6/cmip6-pgf-ingestion-test/zarr_store...
4,HighResMIP,MOHC,HadGEM3-GC31-MM,hist-1950,r1i1p1f1,SImon,siconc,gn,none,r1i1p1f1,v20170928,gs://cmip6/cmip6-pgf-ingestion-test/zarr_store...
5,HighResMIP,NOAA-GFDL,GFDL-CM4C192,hist-1950,r1i1p1f1,SImon,siconc,gn,none,r1i1p1f1,v20180701,gs://cmip6/CMIP6/HighResMIP/NOAA-GFDL/GFDL-CM4...


 ### 7. Preprocess all called models
 Preprocesesed with xMIP - https://cmip6-preprocessing.readthedocs.io/en/latest/?badge=latest

In [9]:
ddict = cat.to_dataset_dict(
    preprocess=combined_preprocessing,
    xarray_open_kwargs={'use_cftime':True},
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


### 8. List all models that have been preprocessed

In [10]:
# list of preprocessed models
mlist = list(ddict.keys())
lct = len(mlist)
print(f'{lct} Models Found')
mlist

6 Models Found


['HighResMIP.CMCC.CMCC-CM2-VHR4.hist-1950.SImon.gn',
 'HighResMIP.CMCC.CMCC-CM2-HR4.hist-1950.SImon.gn',
 'HighResMIP.MOHC.HadGEM3-GC31-LL.hist-1950.SImon.gn',
 'HighResMIP.MOHC.HadGEM3-GC31-HM.hist-1950.SImon.gn',
 'HighResMIP.MOHC.HadGEM3-GC31-MM.hist-1950.SImon.gn',
 'HighResMIP.NOAA-GFDL.GFDL-CM4C192.hist-1950.SImon.gn']

### 9. Reproject and calculate MEF Metric
For each model reproject onto reference grid using xESMF (https://pavics-sdi.readthedocs.io/en/latest/notebooks/regridding.html) <br>
Using the reprojected data calculate the MEF Metric<br>
    - Annual temporal mean<br>
    - Seasonal temporal mean<br>
    - Monthly temporal mean<br>
    - Time serues of spatial mean<br>
All metrics for a model are saved to a dataset then each of those datasets combined into a single compiled dataset.

In [11]:
s_time = time.time()
source_id=[]
for k in range(0,lct,1):
    name = mlist[k].split('.')
    mod_id = name[2]
    source_id.append(mod_id)

MEF=[]
for j in range(0,lct,1):
    # %% Import Model Data (SOURCE)
    model_ds = ddict[mlist[j]]
    model_ds = model_ds.sel(time=slice('1978-12','2014-11'))  # Time slice of data
    model_ds['siconc'] = model_ds['siconc'].where(model_ds['siconc'] <= 100, other=float('nan'))   # Set all flagged values to nan
    dims_to_keep = ['time', 'y', 'x','vertex','bnds']  # Dimensions to be kept
    model_ds = model_ds.squeeze(dim=[dim for dim in model_ds.dims if dim not in dims_to_keep])
    model = source_id[j]

    # %% TARGET - REFERENCE
    # Visualization of input data and its corresponding grid
    bbox = dict(lon_bnds=[-180, 180], lat_bnds=[-80, -50])
    ds_tgt = subset_bbox(ref_ds, **bbox)

    reg_bil = xe.Regridder(model_ds, ds_tgt, "bilinear", periodic=True,)
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Apply the regridding weights to the input sea ice concentration data
    sic_bil = reg_bil(model_ds.siconc)
    model_rp = xr.Dataset({"siconc": sic_bil})

    # Overlay land mask from observational ancillary file
    tct = model_rp.coords['time'].size
    for i in range(0,tct,1):
        model_rp.siconc[i,:,:] = model_rp.siconc[i,:,:]*mask

    model_ds = model_rp.assign_coords(time=obs_ds['time'])  # Set time coords of model ds to be the same as the obs ds
    model_ds = model_ds.swap_dims({'tdim': 'time'})
    model_ds = model_ds.swap_dims({'time': 'tdim'})
    model_da = model_ds.siconc / 100    # Select desired variable and scale to match obs scale of 0-1

    # MEF - ANNUAL
    MEF_num = (obs_da-model_da) ** 2       # cell-wise RMSD
    MEF_num = MEF_num.sum(dim=['tdim'])    # Temporal sum of cell-wise RMSD
    MEF_den = (obs_da-o_mean) ** 2         # cell-wise SD
    MEF_den = MEF_den.sum(dim='tdim')      # Temporal sum of cell-wise SD
    MEF_a = (1 - (MEF_num/MEF_den))

    # MEF - MONTHLY
    MEF_num = (obs_da-model_da) ** 2        # cell-wise RMSD
    MEF_num = MEF_num.groupby('time.month').sum(dim=['tdim'])    # Temporal mean of cell-wise RMSD
    MEF_den = (obs_da-o_mean) ** 2          # cell-wise SD
    MEF_den = MEF_den.groupby('time.month').sum(dim='tdim')      # Temporal mean of cell-wise SD
    MEF_m = (1 - (MEF_num/MEF_den))

    # MEF - SEASONAL
    MEF_num = (obs_da-model_da) ** 2        # cell-wise RMSD
    MEF_num = MEF_num.groupby('time.season').sum(dim=['tdim'])    # Temporal mean of cell-wise RMSD
    MEF_den = (obs_da-o_mean) ** 2          # cell-wise SD
    MEF_den = MEF_den.groupby('time.season').sum(dim='tdim')      # Temporal mean of cell-wise SD
    MEF_s = (1 - (MEF_num/MEF_den))

    # MEF - BINNED
    MEF_num = (obs_da-model_da) ** 2
    MEF_num = MEF_num.sum(dim=['x','y'])
    MEF_den = (obs_da-o_mean) ** 2 
    MEF_den = MEF_den.sum(dim=['x','y'])
    MEF_b = (1 - (MEF_num/MEF_den))

    dataset = xr.Dataset({'MEF_a': MEF_a, 'MEF_s': MEF_s, 'MEF_m': MEF_m, 
                          'MEF_b': MEF_b, 'model_name': model})
   
    MEF.append(dataset)
    print(f"Completed {j + 1} of {lct}")

combined_ds = xr.concat(MEF, dim='models', coords='minimal')
e_time = time.time(); t_time = e_time - s_time
print('Done!')
print(f"Total time taken: {t_time:.2f} seconds")

Completed 1 of 6
Completed 2 of 6
Completed 3 of 6
Completed 4 of 6
Completed 5 of 6
Completed 6 of 6
Done!
Total time taken: 188.12 seconds


/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: 

### 10. Export Dataset
Export cumulative dataset to defined path and name.

In [12]:
s_time = time.time()
print('Initiated')

# combined_ds.to_netcdf(f'CMIP-MEF/CMIP-{lct}-MEF_DEF.nc')
combined_ds.to_netcdf(f'HRMIP-MEF/HRMIP-{lct}-MEF_I80100.nc')

e_time = time.time(); t_time = e_time - s_time
print('Done!')
print(f"Total time taken: {t_time:.2f} seconds")
print(combined_ds.sizes)

Initiated


/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: 

Done!
Total time taken: 375.94 seconds
Frozen({'models': 6, 'y': 332, 'x': 316, 'season': 4, 'month': 12, 'tdim': 432})


/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/opt/anaconda3/envs/pangeo/lib/python3.12/site-packages/dask/core.py:133: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
